In [0]:
!pip install PyDrive

**Imports**

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg
import pandas as pd
import io
import numpy as np
import regex as re
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip '/content/drive/My Drive/validation-set.zip'
!unzip '/content/drive/My Drive/training-set.zip'
#!unzip '/content/drive/My Drive/ImageCLEF_test-set.zip'


In [0]:
mydf = pd.read_csv('train_concepts.csv')
valdf = pd.read_csv('val_concepts.csv')

In [36]:
#Let's print the four first rows of the dataframe, to get an idea of how the df looks like
mydf.head(4)

,image,tags
0,ROCO_CLEF_07350,C0203126;C0203051
1,ROCO_CLEF_19073,C0772294;C0023884;C0221198;C0412555;C0041618
2,ROCO_CLEF_60501,C0233492;C2985494;C0262950;C1306232
3,ROCO_CLEF_05564,C0521530;C0817096


In [0]:
def listifier(mydf):
#Let's turn the tag values from string with internal ';' seperation, to actual lists, and set the image name to index
  for i in range(len(mydf['tags'])):
    mydf['tags'][i] = mydf['tags'][i].split(';')
  return mydf
list_mydf = listifier(mydf)
list_valdf = listifier(valdf)

In [0]:
#We create a list containing all the training imagefiles
train_imagelist = os.listdir('./training-set')
train_imagelist = [i for i in train_imagelist if i.endswith('.jpg')]

In [0]:
img = []
for i in range(10):
    train_list = [train_imagelist[i], mpimg.imread("./training-set/"+train_imagelist[i])]
    img.append(train_list)

In [0]:
newfig = plt.figure(figsize=(20,40))
for pic in range(10):
    plt.subplot(10, 2, pic+1)
    plt.imshow(img[pic][1])
    plt.title(img[pic][0])

In [0]:
#Let's import the tag id-name mapper
tagdf = pd.read_csv('string_concepts.csv', header=None)
#Now let's seperate the tag id and the tag name, name the columns accordingly,set the id as index and prin a few rows.
tagdf = tagdf[tagdf.columns[0]].str.split(pat = '\t',n=0,  expand=True)
tagdf.columns = ['tagid', 'tagname']
tagdf = tagdf.set_index('tagid')
tagdf.head(5)

In [0]:
#Now let's print the tags ids and names of the images we plotted
for i in range(10):
  print(mydf['image'][i])
  for j in mydf['tags'][i]:
    print('\t' + j + ' = ' + tagdf['tagname'][j])

In [0]:
#Let's get the total count of tags and the count of unique tags and the 10 most frequent terms.
#We also prin the avg number of tags per image
tagunique = []
taglist = []
for i in range(len(mydf['image'])):
  for j in mydf['tags'][i]:
    if j not in tagunique:
      tagunique.append(j)
    taglist.append(j)
print("There is a total of " + str(len(tagunique)) + " unique values, assigned in total " + str(len(taglist)) + " times \n" )

from collections import Counter
print("The 6 most common tags, along with their number of occurencies are: \n" + str(Counter(taglist).most_common(6)) +'\n')
print("The average number of tags per image is " + str(len(taglist)/len(mydf['image'])))

##One-hot

###One-hot Encoder. Failed attempt



The following attempt would cause an issue of assigning the values to the followin column. This was inherented by the transformation of the numpy array following the encoding, to df. Possible cause of the issue is the reservation of the 0 position by tensorflow. Attempts to reassign, caused shape mismatches among the structures used, or wrong results for the first/last attribute columns.

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

token_index = []
for i in mydf['tags']:
  for j in i:
    if j not in token_index:
      token_index.append(j)
newar = mydf['tags'].to_numpy()
tokenizer = Tokenizer(num_words=(len(token_index)))
tokenizer.fit_on_texts(newar)
one_hot_results = tokenizer.texts_to_matrix(newar, mode='binary')

#tokenizer alphabetically orders the column names, but keeps the initial row order it was given
#So, we can simply concat the numpy array outputed with the dataframe we created, including the name and the attributes list of the images
newdf = pd.DataFrame(data = one_hot_results, columns = list(tokenizer.word_index.keys()))
result1 = pd.concat([mydf, newdf], axis=1, sort=False)

In [0]:
#Check for assignment validity. Expected 1s
for i in range(1,3):
  for j in result1['tags'][i]:
    j = j.lower()
    #print(result[j][i])
    print("Score for " + str(result1['tags'][i]) + " in tag " + j + ' is ' + str(result1[j][i]))

###One-hot Encoder. Accepted alternative

In [42]:
def onehotenc(mydf):
  mlb = MultiLabelBinarizer()
  multibinarray = mlb.fit_transform(mydf['tags'])
  mydf1 = pd.DataFrame(data = multibinarray, columns = mlb.classes_)
  result = pd.concat([mydf, mydf1], axis=1, sort=False)
  #Let's try a few examples to make sure our values are correctly assigned after the conact. We are expecting all 1s
  for i in range(1,3):
    for j in result['tags'][i]:
      print("Score for " + str(result['tags'][i]) + " in tag " + j + ' is ' + str(result[j][i]) )
  return(result)
#Before implementing our one-hot encoding, we need to append the train and validation set
#This is in order to acquire all the available labels, as well as having the same number of label columns in both sets.
fulldf = mydf.append(valdf, ignore_index=True)
full_result = onehotenc(fulldf)
#We will now create two dataframes, for training and validating data respectively. We will use the columns from the appended one-hoted df,
#and we will assign to each the accroding data. Given the fact that the order of the rows remain unchanged, we will use the length 
#parameters to split the data to the two new dfs. 
hot_train = pd.DataFrame(columns=full_result.columns)
hot_valid = pd.DataFrame(columns=full_result.columns)
hot_train = full_result.head(len(list_mydf))
hot_valid = full_result.iloc[-len(list_valdf):]
hot_valid = hot_valid.reset_index(drop = True)
#train_result = onehotenc(mydf)
#valid_result = onehotenc(valdf)

Score for ['C0772294', 'C0023884', 'C0221198', 'C0412555', 'C0041618'] in tag C0772294 is 1
Score for ['C0772294', 'C0023884', 'C0221198', 'C0412555', 'C0041618'] in tag C0023884 is 1
Score for ['C0772294', 'C0023884', 'C0221198', 'C0412555', 'C0041618'] in tag C0221198 is 1
Score for ['C0772294', 'C0023884', 'C0221198', 'C0412555', 'C0041618'] in tag C0412555 is 1
Score for ['C0772294', 'C0023884', 'C0221198', 'C0412555', 'C0041618'] in tag C0041618 is 1
Score for ['C0233492', 'C2985494', 'C0262950', 'C1306232'] in tag C0233492 is 1
Score for ['C0233492', 'C2985494', 'C0262950', 'C1306232'] in tag C2985494 is 1
Score for ['C0233492', 'C2985494', 'C0262950', 'C1306232'] in tag C0262950 is 1
Score for ['C0233492', 'C2985494', 'C0262950', 'C1306232'] in tag C1306232 is 1


In [43]:
hot_valid.head(3)

,image,tags,C0000503,C0000723,C0000726,C0000780,C0000782,C0000790,C0000828,C0000925,C0000934,C0000962,C0001074,C0001162,C0001209,C0001416,C0001428,C0001430,C0001511,C0001558,C0001563,C0001575,C0001577,C0001613,C0001629,C0001632,C0001701,C0001721,C0001811,C0002045,C0002191,C0002374,C0002638,C0002688,C0002691,C0002708,C0002766,C0002771,C0002844,C0002873,...,C4283785,C4283791,C4283904,C4284399,C4284930,C4284931,C4285062,C4285610,C4288160,C4288293,C4288599,C4309954,C4315900,C4316796,C4316915,C4317146,C4318483,C4318616,C4318652,C4318935,C4319192,C4319531,C4321236,C4321335,C4321352,C4321395,C4321408,C4321499,C4329495,C4329648,C4330491,C4331246,C4331837,C4476838,C4479170,C4479578,C4505257,C4505346,C4505362,C4505478
0,ROCO_CLEF_64017,"[C0001074, C3203359, C0041618]",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ROCO_CLEF_39073,"[C0027530, C1610719, C0065967, C1704258, C0883...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ROCO_CLEF_18028,"[C1265876, C1293134, C0029053, C0034579, C0025...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
def jpgfier(plaindf):
  #Adds the .jpg suffix, as the flow from dataframe function requires the input filename to include it.
  pd.set_option('mode.chained_assignment', None)
  for i in range(len(plaindf['image'])):
    plaindf['image'][i] = str(plaindf['image'][i]+'.jpg')
  return plaindf
train_result = full_result.head(len(mydf))
train_result = jpgfier(hot_train)
valid_result = jpgfier(hot_valid)

##Data load
Obviously, importing the whole image datasets based on our raw muscle, will not even work in a tpu-accelerated env. We could try manually resizing and batching, but we can simply use the ImageDataGenerator, which will also be our base for data augmentation happening later on.


###Data Preperation. Alternative #1
The following alternative does not require the execution of the one-hot encoder implemented eariler. It offers an extra level of security iver the outpouted result, as not only the number of rows, but the number of classes identified is given as info created by the process. 

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1/255.)

trainit=datagen.flow_from_dataframe(dataframe=train_result,directory="./training-set/",x_col="image",
                                    y_col=train_result.columns[2:],class_mode="other",target_size=(100,100))

validit=datagen.flow_from_dataframe(dataframe=valid_result,directory="./validation-set/",x_col="image",
                                    y_col=valid_result.columns[2:],class_mode="other",target_size=(100,100))


Found 56629 validated image filenames.
Found 14157 validated image filenames.


###Data Preperation. Alternative #2
The following alternative does not require the execution of the one-hot encoder implemented eariler. It offers an extra level of security iver the outpouted result, as not only the number of rows, but the number of classes identified is given as info created by the process. 

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1/255.)
trainit=datagen.flow_from_dataframe( dataframe=train_result, directory="./training-set/",x_col="image", 
                                    y_col=train_result.columns[1], class_mode="categorical", target_size=(100,100))

validit=datagen.flow_from_dataframe( dataframe=valid_result, directory="./validation-set/",x_col="image", 
                                    y_col=valid_result.columns[1], class_mode="categorical", target_size=(100,100))


Found 14157 validated image filenames belonging to 3233 classes.


##Baseline 
As our baseline we will leverage the dummygenerator, having as strategy the stratified 

In [0]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
trainx = train_result['image']
trainy = train_result.iloc[:,2:].to_numpy()
validx = valid_result['image']
dummy_clf = DummyClassifier(strategy="stratified")
validy = valid_result.iloc[:,2:].to_numpy()
dummy_clf.fit(trainx,trainy)
yres = dummy_clf.predict(validx)
f1_score(validy, yres)
f1_score(validy, yres, average='micro')

##First neural model. 
Plain model from scratch

In [47]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))

Using TensorFlow backend.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
